In [14]:
from sklearn.cross_decomposition import CCA
import warnings
warnings.filterwarnings('ignore')
import xarray as xr
import numpy as np
import gc
import pandas as pd
import datetime as dt
import dask
from datetime import datetime
import matplotlib.pyplot as plt
import os
import s3fs
import zarr
import geocat.comp
import tensorflow as tf
from sklearn.metrics import roc_curve, auc
from sklearn import preprocessing
from math import e


In [15]:
#Preprocessing functions
#3D detrend function
def detrend(x:np.ndarray,time:np.ndarray):
        nt,nx,ny = x.shape
        xtemp = x.reshape(nt,nx*ny)
        p = np.polyfit(time, xtemp, deg=3)
        fit = p[0]*(time[:,np.newaxis] **3)+ p[1]*(time[:,np.newaxis]**2) + p[2]*(time[:,np.newaxis]) + p[3]
        return x - fit.reshape(nt,nx,ny)
    
#1D detrend function
def altdetrend(x:np.ndarray,time:np.ndarray):
        nt = x.shape
        xtemp = x.reshape(nt)
        p = np.polyfit(time, x, deg=1)
        fit = p[0]*(time[:,np.newaxis])+ p[1]
        return x - fit.reshape(nt)
    
def remove_time_mean(x):
        return x - x.mean(dim='time')

def removeSC(x):
        return x.groupby('time.month').apply(remove_time_mean)

# Calculate std normal anomaly
def calStdNorAnom(x):
    a=[]
    for m in np.unique(x.time.dt.month):
        mData=x[x.time.dt.month==m]
        mRolling=mData.rolling(time=31, center=True).mean().bfill(dim="time").ffill(dim="time")
        sRolling=mData.rolling(time=31, center=True).std().bfill(dim="time").ffill(dim="time")
        normData=(mData-mRolling)/sRolling
        a.append(normData)
    combineArray=xr.concat(a,'time')
    outArray=combineArray.sortby('time')
    return outArray

In [16]:
# Connect to AWS S3 storage
fs = s3fs.S3FileSystem(anon=True)
## By downloading the master CSV file enumerating all available data stores, we can interact with the spreadsheet
## through a pandas DataFrame to search and explore for relevant data using the CMIP6 controlled vocabulary:
df = pd.read_csv("https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.csv")

In [4]:
def getData(qstring):
    df_subset = df.query(qstring)
    if df_subset.empty:
        print('data not available for '+qstring)
    else:
        for v in df_subset.zstore.values:
            zstore = v
            mapper = fs.get_mapper(zstore)
            return_ds = xr.open_zarr(mapper, consolidated=True)
    return(return_ds)

In [5]:
#Open CESM2 Ensemble Datasets
modelName="'CESM2'"
institute="'NCAR'"
expList=["'historical'"]
actList=["'CMIP'"]
membList=["'r1i1p1f1'", "'r2i1p1f1'", "'r3i1p1f1'", "'r4i1p1f1'" , "'r5i1p1f1'", "'r6i1p1f1'", "'r7i1p1f1'","'r8i1p1f1'","'r9i1p1f1'","'r10i1p1f1'","'r11i1p1f1'"]

In [6]:
files='Regrided_ERA5.nc'
#Open ERA5 Dataset
data=xr.open_dataset(files)
cres=data.cres_Pre
pr=data.pr
land=data.lsMask


#Select Monsoon Months
months=[6,7,8,9]
lagmonths=[6,7,8,9]
#lagmonths=[2,3,4,5]
#months=[8]
#lagmonths=[5]
#varOut.where(varOut.time.dt.month.isin(months), drop=True) #Change varOut to desired variable
prec=pr.where(pr.time.dt.month.isin(months), drop=True)
land=land.where(land.time.dt.month.isin(months), drop=True)
cres=cres.where(cres.time.dt.month.isin(lagmonths), drop=True)


#Select only the SAM lat,lon range: 60-100E, 10-30N
precip=prec.sel(lon=slice(60,100),lat=slice(10,30))
land=land.sel(lon=slice(60,100),lat=slice(10,30))
precip=xr.where(land==0,np.nan,precip) #remove oceans, monsoon is defined as only over land 

#Do weighted correction on precipitation
weights=np.cos(np.deg2rad(precip.lat))
prec_index=precip.weighted(weights).mean(dim=('lat','lon'))
prec_index=prec_index*60*60*24 #conversion to mm/day, exluding dividing by rho and multiplying by 1000mm/m

#Remove seasonal cycle
prec_index=removeSC(prec_index)

#Normalize
prec_index=calStdNorAnom(prec_index)

#Detrend data sets
time=prec_index.time
prec_index=prec_index.to_numpy()
time=time.to_numpy()
time=time.astype(int)/10**9

prec_index=altdetrend(prec_index,time)
prec_index=xr.DataArray(prec_index,coords=[time],dims=['time'])


pr_new,cres = xr.broadcast(prec_index,cres) #broadcast pr_mean array to fill array to allow regression to be executed

pr_new=pr_new.dropna(dim='time')
cres=cres.dropna(dim='time')

In [7]:
cca=CCA(n_components=1)
testcca=[]
for i in range(len(cres.time)):
    for j in range (len(pr_new.time)):
        U_c , V_c =cca.fit_transform(cres[i],pr_new[j])
        
        result=np.corrcoef(U_c.T,V_c.T)[0,1]
        
        #print(result)
        
        testcca.append(result)
        

In [13]:
cres.shape

(172, 181, 240)

In [9]:
len(testcca)

29584

In [ ]:
cres=cres.to_numpy()
ntime, nlat, nlon = cres.shape

d2_cres=cres.reshape((ntime,nlat*nlon))

d2_cres.shape

In [ ]:
cca=CCA(n_components=1)

x_c, y_c =cca.fit_transform(d2_cres,yIn)


result = np.corrcoef(x_c.T, y_c.T)#[0,1]


In [ ]:
creslist=[]
crellist=[]
netTOAcslist=[]
dummy_ylist=[]

for j,memb in enumerate(membList):
        inputStr  = "institution_id =='NCAR' & source_id=='CESM2' & table_id=='Amon' & experiment_id=='historical' &  member_id=="+memb+" & variable_id=="
        altinputStr  = "activity_id=='CMIP' & table_id=='fx' & source_id=='CESM2' & experiment_id=='historical' &  member_id=="+memb+" & variable_id=="
        rsut_ds   = getData(inputStr+"'rsut'")
        pr_ds     = getData(inputStr+"'pr'")
        rsutcs_ds = getData(inputStr+"'rsutcs'")
        sftlf_ds = getData(altinputStr+"'sftlf'")
        rlut_ds   =getData(inputStr+"'rlut'")
        rlutcs_ds  =getData(inputStr+"'rlutcs'")
        rsdt_ds  =getData(inputStr+"'rsdt'")
        
        netTOAcs = rsdt_ds.rsdt - rsutcs_ds.rsutcs - rlutcs_ds.rlutcs
        
        cres= rsutcs_ds.rsutcs-rsut_ds.rsut
        crel=rlutcs_ds.rlutcs-rlut_ds.rlut
        prec=pr_ds.pr
        land=sftlf_ds.sftlf
    
        datetimeindex=cres.indexes['time'].to_datetimeindex()
        cres['time']=datetimeindex
        crel['time']=datetimeindex
        netTOAcs['time']=datetimeindex
        prec['time']=datetimeindex
        
        cres,land= xr.broadcast(cres,land) #add time dimension to land variable for compatability with CRE variables
        
        
        
        months=[6,7,8,9]
        leadmonths=[6,7,8,9]

        #varOut.where(varOut.time.dt.month.isin(months), drop=True) #Change varOut to desired variable
        prec=prec.sel(time=prec.time.dt.month.isin(months))
        cres=cres.sel(time=cres.time.dt.month.isin(leadmonths))
        crel=crel.sel(time=crel.time.dt.month.isin(leadmonths))
        netTOAcs=netTOAcs.sel(time=netTOAcs.time.dt.month.isin(leadmonths))
        land=land.sel(time=land.time.dt.month.isin(months))
        
        #Select only the SAM lat,lon range: 60-100E, 10-30N
        precip=prec.sel(lon=slice(60,100),lat=slice(10,30))
        land=land.sel(lon=slice(60,100),lat=slice(10,30))
        
        precip=xr.where(land==0,np.nan,precip) #remove oceans, monsoon is defined as only over land 
        
        #Do weighted correction on precipitation
        weights=np.cos(np.deg2rad(precip.lat))
        prec_index=precip.weighted(weights).mean(dim=('lat','lon'))
        prec_index=prec_index*60*60*24 #conversion to mm/day, exluding dividing by rho and multiplying by 1000mm/m
        
        lat=cres.lat
        lon=cres.lon
        
        #Do the preprocessing
        cres=removeSC(cres)
        cres=calStdNorAnom(cres)
        
        crel=removeSC(crel)
        crel=calStdNorAnom(crel)
        
        netTOAcs=removeSC(netTOAcs)
        netTOAcs=calStdNorAnom(netTOAcs)
        
        #Detrend
        time=cres.time
        cres=cres.to_numpy()
        crel=crel.to_numpy()
        netTOAcs=netTOAcs.to_numpy()
        time=time.to_numpy()
        time=time.astype(int)/10**9
        
        cres=detrend(cres,time)
        cres=xr.DataArray(cres,coords=[time,lat,lon],dims=['time','lat','lon'])
        
        crel=detrend(crel,time)
        crel=xr.DataArray(crel,coords=[time,lat,lon],dims=['time','lat','lon'])
        
        netTOAcs=detrend(netTOAcs,time)
        netTOAcs=xr.DataArray(netTOAcs,coords=[time,lat,lon],dims=['time','lat','lon'])
        
        #Precip preprocessing
        #Remove seasonal cycle
        prec_index=removeSC(prec_index)
        
        #Normalize
        prec_index=calStdNorAnom(prec_index)
        
        #Detrend
        time=prec_index.time
        prec_index=prec_index.to_numpy()
        time=time.to_numpy()
        time=time.astype(int)/10**9
        
        prec_index=altdetrend(prec_index,time)
        prec_index=xr.DataArray(prec_index,coords=[time],dims=['time'])
        
        mysd=prec_index.std()
        mymean=prec_index.mean()

        buckets=pd.Categorical(pd.cut(prec_index, [mymean - mysd* 10000,  mymean - 0.5*mysd,  mymean + 0.5*mysd, mymean + mysd* 10000])).rename_categories(['low','average','high'])

        le = preprocessing.LabelEncoder()
        le.fit(buckets)

        labelprec=le.transform(buckets)

        # convert integers to dummy variables (i.e. one hot encoded)
        nclasses=3
        dummy_y=to_categorical(labelprec,nclasses) #converts to binary
        
        creslist.append(cres)
        cresstack=np.stack(creslist,axis=0)
        
        crellist.append(crel)
        crelstack=np.stack(crellist,axis=0)
        
        netTOAcslist.append(netTOAcs)
        netTOAcsstack=np.stack(netTOAcslist,axis=0)
        
        dummy_ylist.append(dummy_y)
        dummy_ystack=np.stack(dummy_ylist,axis=0)
              

In [ ]:
#Seperate each cres array
cresIn1=cresstack[0,:,:,:]
cresIn2=cresstack[1,:,:,:]
cresIn3=cresstack[2,:,:,:]
cresIn4=cresstack[3,:,:,:]
cresIn5=cresstack[4,:,:,:]
cresIn6=cresstack[5,:,:,:]
cresIn7=cresstack[6,:,:,:]
cresIn8=cresstack[7,:,:,:]
cresIn9=cresstack[8,:,:,:]
cresIn10=cresstack[9,:,:,:]
cresIn11=cresstack[10,:,:,:]


#Concatenate the ensemble members along time axis
cresIn=np.concatenate((cresIn1,cresIn2,cresIn3,cresIn4,cresIn5,cresIn6,cresIn7,cresIn8,cresIn9,cresIn10,cresIn11),axis=0)

#Seperate each crel array
crelIn1=crelstack[0,:,:,:]
crelIn2=crelstack[1,:,:,:]
crelIn3=crelstack[2,:,:,:]
crelIn4=crelstack[3,:,:,:]
crelIn5=crelstack[4,:,:,:]
crelIn6=crelstack[5,:,:,:]
crelIn7=crelstack[6,:,:,:]
crelIn8=crelstack[7,:,:,:]
crelIn9=crelstack[8,:,:,:]
crelIn10=crelstack[9,:,:,:]
crelIn11=crelstack[10,:,:,:]

#Concatenate the ensemble members along time axis
crelIn=np.concatenate((crelIn1,crelIn2,crelIn3,crelIn4,crelIn5,crelIn6,crelIn7,crelIn8,crelIn9,crelIn10,crelIn11),axis=0)

#Seperate each netTOAcs array
netTOAcsIn1=netTOAcsstack[0,:,:,:]
netTOAcsIn2=netTOAcsstack[1,:,:,:]
netTOAcsIn3=netTOAcsstack[2,:,:,:]
netTOAcsIn4=netTOAcsstack[3,:,:,:]
netTOAcsIn5=netTOAcsstack[4,:,:,:]
netTOAcsIn6=netTOAcsstack[5,:,:,:]
netTOAcsIn7=netTOAcsstack[6,:,:,:]
netTOAcsIn8=netTOAcsstack[7,:,:,:]
netTOAcsIn9=netTOAcsstack[8,:,:,:]
netTOAcsIn10=netTOAcsstack[9,:,:,:]
netTOAcsIn11=netTOAcsstack[10,:,:,:]

#Concatenate the ensemble members along time axis
netTOAcsIn=np.concatenate((netTOAcsIn1,netTOAcsIn2,netTOAcsIn3,netTOAcsIn4,netTOAcsIn5,netTOAcsIn6,netTOAcsIn7,netTOAcsIn8,netTOAcsIn9,netTOAcsIn10,netTOAcsIn11),axis=0)

#Seperate each precip_index array
yIn1=dummy_ystack[0,:,:]
yIn2=dummy_ystack[1,:,:]
yIn3=dummy_ystack[2,:,:]
yIn4=dummy_ystack[3,:,:]
yIn5=dummy_ystack[4,:,:]
yIn6=dummy_ystack[5,:,:]
yIn7=dummy_ystack[6,:,:]
yIn8=dummy_ystack[7,:,:]
yIn9=dummy_ystack[8,:,:]
yIn10=dummy_ystack[9,:,:]
yIn11=dummy_ystack[10,:,:]

#Concatenate the ensemble members along time axis
yIn=np.concatenate((yIn1,yIn2,yIn3,yIn4,yIn5,yIn6,yIn7,yIn8,yIn9,yIn10,yIn11),axis=0)